In [2]:
from sentence_transformers import SentenceTransformer
import pinecone
import numpy as np

In [5]:
pinecone.init(api_key="91e41499-ee04-4431-8ddc-d878f12a62a0", environment="us-west4-gcp-free")

In [6]:
pinecone.create_index("chatbot", dimension=384, metric="cosine")

In [25]:
pinecone.list_indexes()
index = pinecone.Index("chatbot")

In [9]:
sentences =  [
    "What is Foliogrow",
    "Who are the software partners",
    "What is the contact address",
    
    ]

# answers = [
#     "FolioGrow is smart cannabis cultivation software that helps you make the right decisions to increase your yields and profits",
#     "metrc,ridder,growlink,netexam,salesforce,IBMwatson",
#     " 3002 Commerce Street Dallas, TX 75226"
# ]

# urls = [
#     "https://foliogrow.com/",
#     "https://foliogrow.com/metrc/",
#     "https://foliogrow.com/contact/"
# ]

In [36]:
model = SentenceTransformer('./all-MiniLM-L6-v2')
embeddings = []
for x in sentences:
    embedding = model.encode(x)
    embeddings.append(embedding)
embeddings_array = np.array(embeddings)
print(embeddings_array)
print(len(embeddings_array[0]))
    





[[-0.01210796  0.03986302 -0.11864929 ... -0.04804447  0.09362378
   0.03605172]
 [-0.05488758 -0.03095406 -0.01984533 ... -0.07090454  0.09768245
   0.01130756]
 [-0.13744348 -0.00276467  0.03286193 ...  0.03753722 -0.04565042
   0.02908565]]
384


In [37]:
embeddings_array =  embeddings_array.tolist()


In [38]:
index.upsert([
    ("A", embeddings_array[0], {"answer": "FolioGrow is smart cannabis cultivation software that helps you make the right decisions to increase your yields and profits", "url": "https://foliogrow.com/"}),
    ("B", embeddings_array[1], {"answer": "metrc,ridder,growlink,netexam,salesforce,IBMwatson", "url": "https://foliogrow.com/metrc/"}),
    ("C", embeddings_array[2], {"answer": "3002 Commerce Street Dallas, TX 75226", "url": "https://foliogrow.com/metrc/"}),
    
])

{'upserted_count': 3}

In [40]:
index.query(
  vector=embeddings_array[2],
  top_k=3,
  include_values=False,
  include_metadata=True
)

{'matches': [{'id': 'C',
              'metadata': {'answer': '3002 Commerce Street Dallas, TX 75226',
                           'url': 'https://foliogrow.com/metrc/'},
              'score': 1.0,
              'values': []},
             {'id': 'B',
              'metadata': {'answer': 'metrc,ridder,growlink,netexam,salesforce,IBMwatson',
                           'url': 'https://foliogrow.com/metrc/'},
              'score': 0.117907159,
              'values': []},
             {'id': 'A',
              'metadata': {'answer': 'FolioGrow is smart cannabis cultivation '
                                     'software that helps you make the right '
                                     'decisions to increase your yields and '
                                     'profits',
                           'url': 'https://foliogrow.com/'},
              'score': 0.0476765,
              'values': []}],
 'namespace': ''}